In [1]:
!wget -nc https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos/pub/FEAGA_FEADER_2014.zip

File ‘FEAGA_FEADER_2014.zip’ already there; not retrieving.



In [2]:
!wget -nc https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos/pub/FEAGA_FEADER_2015.zip

--2017-05-17 20:27:31--  https://aplj13.ifap.min-agricultura.pt/idigital.consultarFundos/pub/FEAGA_FEADER_2015.zip
Resolving aplj13.ifap.min-agricultura.pt... 91.216.117.2
Connecting to aplj13.ifap.min-agricultura.pt|91.216.117.2|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7179124 (6.8M) [application/zip]
Saving to: ‘FEAGA_FEADER_2015.zip’

FEAGA_FEADER_2015.z 100%[===================>]   6.85M   749KB/s    in 9.5s    

2017-05-17 20:27:41 (740 KB/s) - ‘FEAGA_FEADER_2015.zip’ saved [7179124/7179124]



In [20]:
import pandas as pd
import numpy as np

In [24]:
df_2015 = pd.read_csv(f'FEAGA_FEADER_2015.zip', delimiter=';', encoding='latin1', skiprows=1, compression='zip')
df_2015 = df_2015.rename(columns={
        'exercicio': 'year',
        'medida': 'scheme',
        'montante': 'amount',
        'denominacao': 'recipient_name',
        'codigo_postal': 'recipient_postcode',
    })
df_2015['distrito'] = pd.to_numeric(df_2015['distrito'], errors='coerce')
df_2015['concelho'] = pd.to_numeric(df_2015['concelho'], errors='coerce')

distrito_dict = df_2015.groupby(['distrito']).first()['descricao_distrito'].to_dict()
concelho_dict = df_2015.groupby(['concelho']).first()['descricao_concelho'].to_dict()
df_2015['distrito'] = df_2015['distrito'].map(distrito_dict)
df_2015['concelho'] = df_2015['concelho'].map(concelho_dict)
df_2015 = df_2015.drop(['descricao_distrito', 'descricao_concelho'], 1)
df_2015.head()

/Users/sw/.virtualenvs/notebook36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2683: DtypeWarning: Columns (2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,year,recipient_name,distrito,concelho,recipient_postcode,scheme,amount
0,2015,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,III.6,"2029,99"
1,2015,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,IV/A.18,"2063,4"
2,2015,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,IV/B.1.6,"5476,48"
3,2015,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,IV/B.2.4,"836,32"
4,2015,A & D SERVIÇOS E INVESTIMENTOS LDA,PORTO,BAIAO,4640,I.1,"1370,16"


In [25]:
df_2014 = pd.read_csv(f'FEAGA_FEADER_2014.zip', delimiter=';', encoding='latin1', skiprows=1, compression='zip')
df_2014 = df_2014.rename(columns={
        u'Exercício': 'year',
        'Medida': 'scheme',
        'Montante': 'amount',
        'Distrito': 'distrito',
        'Concelho': 'concelho',
        u'Denominação': 'recipient_name',
        u'Código Postal': 'recipient_postcode',
    })

df_2014['distrito'] = pd.to_numeric(df_2014['distrito'], errors='coerce')
df_2014['concelho'] = pd.to_numeric(df_2014['concelho'], errors='coerce')
df_2014['distrito'] = df_2014['distrito'].map(distrito_dict)
df_2014['concelho'] = df_2014['concelho'].map(concelho_dict)

df_2014.head()

,year,recipient_name,distrito,concelho,recipient_postcode,scheme,amount
0,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",AVEIRO,NaN,9900,IV/B.1.8,"34378,05"
1,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",AVEIRO,NaN,9900,IV/B.4.3,"11480,35"
2,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",AVEIRO,NaN,9900,Total,"45858,4"
3,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,BEJA,SERPA,4640,III.6,"3211,56"
4,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,BEJA,SERPA,4640,IV/B.1.6,"7002,61"


In [26]:
df = pd.concat([df_2014, df_2015])
df['concelho'] = df['concelho'].fillna('')
df['distrito'] = df['distrito'].fillna('')
df.head()

,year,recipient_name,distrito,concelho,recipient_postcode,scheme,amount
0,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",AVEIRO,,9900,IV/B.1.8,"34378,05"
1,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",AVEIRO,,9900,IV/B.4.3,"11480,35"
2,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",AVEIRO,,9900,Total,"45858,4"
3,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,BEJA,SERPA,4640,III.6,"3211,56"
4,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,BEJA,SERPA,4640,IV/B.1.6,"7002,61"


In [27]:
df['recipient_location'] = df.apply(lambda x: u'%s, %s' % (x['concelho'] or '', x['distrito']), 1)
df['amount'] = pd.to_numeric(df['amount'].str.replace(',', '.'))
df.head()

,year,recipient_name,distrito,concelho,recipient_postcode,scheme,amount,recipient_location
0,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",AVEIRO,,9900,IV/B.1.8,34378.05,", AVEIRO"
1,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",AVEIRO,,9900,IV/B.4.3,11480.35,", AVEIRO"
2,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",AVEIRO,,9900,Total,45858.40,", AVEIRO"
3,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,BEJA,SERPA,4640,III.6,3211.56,"SERPA, BEJA"
4,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,BEJA,SERPA,4640,IV/B.1.6,7002.61,"SERPA, BEJA"


In [28]:
len(df)

1316779

In [29]:
from slugify import slugify
df['recipient_id'] = df.apply(lambda x: 'PT-%s-%s' % (x['recipient_postcode'], slugify(x['recipient_name'])), 1)
df = df.drop(['distrito', 'concelho'], 1)
df['currency'] = 'EUR'
df['country'] = 'PT'
df.head()

,year,recipient_name,recipient_postcode,scheme,amount,recipient_location,recipient_id,currency,country
0,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",9900,IV/B.1.8,34378.05,", AVEIRO",PT-9900-O-MORRO-FABRICACAO-DE-QUEIJOS-LDA,EUR,PT
1,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",9900,IV/B.4.3,11480.35,", AVEIRO",PT-9900-O-MORRO-FABRICACAO-DE-QUEIJOS-LDA,EUR,PT
2,2014,""" O MORRO "" - FABRICAÇÃO DE QUEIJOS, LDA.",9900,Total,45858.40,", AVEIRO",PT-9900-O-MORRO-FABRICACAO-DE-QUEIJOS-LDA,EUR,PT
3,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,4640,III.6,3211.56,"SERPA, BEJA",PT-4640-A-D-SERVICOS-E-INVESTIMENTOS-LDA,EUR,PT
4,2014,A & D SERVIÇOS E INVESTIMENTOS LDA,4640,IV/B.1.6,7002.61,"SERPA, BEJA",PT-4640-A-D-SERVICOS-E-INVESTIMENTOS-LDA,EUR,PT


In [30]:
df['year'].value_counts()

2015    662420
2014    654359
Name: year, dtype: int64

In [31]:
df.to_csv('pt_2014_2015.csv.gz', index=False, encoding='utf-8', compression='gzip')